In [1]:
import numpy as np
import pandas as pd
import re
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, RepeatedStratifiedKFold
from sklearn.metrics import roc_auc_score, recall_score, classification_report,confusion_matrix, f1_score
import glob
import os
import matplotlib.pyplot as plt

In [2]:
# train_path = max(glob.glob('./data/twitter_data_train/*.csv'), key=os.path.getctime) 
# test_path = max(glob.glob('./data/test/*.csv'), key=os.path.getctime) 
train_df = pd.read_csv('./data/twitter_data_train_multiclass.csv')
test_df = pd.read_csv('./data/twitter_data_test_multiclass.csv')

print(train_df.shape)
print(test_df.shape)

(9445, 235)
(1667, 235)


In [3]:
to_drop = ['screen_name', 'url', 'profile_image_url', 'description',
           'id', 'name', 'account_type','account_type_multi', 'tweets_list', 'tweets_list_processed',
          'description_processed', 'protected', 'verified']

X_train, y_train = train_df.drop(to_drop, axis=1), train_df['account_type_multi']
X_test, y_test = test_df.drop(to_drop, axis=1), test_df['account_type_multi']

In [4]:
X_train.columns[:30]

Index(['Unnamed: 0', 'statuses_count', 'followers_count', 'friends_count',
       'favourites_count', 'listed_count', 'default_profile',
       'default_profile_image', 'geo_enabled', 'profile_use_background_image',
       'profile_background_tile', 'tweet_frequency', 'number_of_tags',
       'number_of_mentions', 'tweet_weekend_frequency',
       'tweet_weekday_frequency', 'following_to_followers_ratio',
       'username_length', 'screen_name_length', 'username_spec_char_count',
       'screen_name_spec_char_count', 'has_url', 'tweets_0', 'tweets_1',
       'tweets_2', 'tweets_3', 'tweets_4', 'tweets_5', 'tweets_6', 'tweets_7'],
      dtype='object')

In [5]:
# Combine train and test data set tgt

X = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

In [6]:
# Check if dataset is balanced
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(9445, 222)
(9445,)
(1667, 222)
(1667,)


In [7]:
# Make sure all data are of integer of float type
X_train.dtypes.unique()

array([dtype('int64'), dtype('float64')], dtype=object)

In [8]:
#Default model
model_xgboost_default = xgb.XGBClassifier(objective='multi:softprob')

model_xgboost_default.fit(X_train,
                  y_train,
                  early_stopping_rounds=10,
                  eval_set=[(X_test, y_test)],
                  verbose=True)

xgb_train_default_predict = model_xgboost_default.predict(X_train)
xgb_test_default_predict = model_xgboost_default.predict(X_test)


c:\Users\bandy\AppData\Local\Programs\Python\Python310\lib\site-packages\xgboost\sklearn.py:793: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mlogloss:1.03431
[1]	validation_0-mlogloss:0.72202
[2]	validation_0-mlogloss:0.53013
[3]	validation_0-mlogloss:0.39909
[4]	validation_0-mlogloss:0.30820
[5]	validation_0-mlogloss:0.24296
[6]	validation_0-mlogloss:0.19545
[7]	validation_0-mlogloss:0.16075
[8]	validation_0-mlogloss:0.13401
[9]	validation_0-mlogloss:0.11343
[10]	validation_0-mlogloss:0.09742
[11]	validation_0-mlogloss:0.08565
[12]	validation_0-mlogloss:0.07629
[13]	validation_0-mlogloss:0.06953
[14]	validation_0-mlogloss:0.06424
[15]	validation_0-mlogloss:0.06047
[16]	validation_0-mlogloss:0.05760
[17]	validation_0-mlogloss:0.05436
[18]	validation_0-mlogloss:0.05272
[19]	validation_0-mlogloss:0.05056
[20]	validation_0-mlogloss:0.04923
[21]	validation_0-mlogloss:0.04810
[22]	validation_0-mlogloss:0.04779
[23]	validation_0-mlogloss:0.04754
[24]	validation_0-mlogloss:0.04695
[25]	validation_0-mlogloss:0.04621
[26]	validation_0-mlogloss:0.04589
[27]	validation_0-mlogloss:0.04577
[28]	validation_0-mlogloss:0.0

In [9]:
print(model_xgboost_default.get_params)

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)>


**Evaluate Model Performance**

In [11]:
print(confusion_matrix(y_train, xgb_train_default_predict))
print(classification_report(y_train, xgb_train_default_predict))

[[2343    0    0    0    0    0]
 [   0 2701    0    0    0    0]
 [   0    0  300    0    0    0]
 [   0    0    0 3706    0    0]
 [   0    0    0    0  316    0]
 [   0    0    0    0    0   79]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2343
           1       1.00      1.00      1.00      2701
           2       1.00      1.00      1.00       300
           3       1.00      1.00      1.00      3706
           4       1.00      1.00      1.00       316
           5       1.00      1.00      1.00        79

    accuracy                           1.00      9445
   macro avg       1.00      1.00      1.00      9445
weighted avg       1.00      1.00      1.00      9445



In [12]:
print(confusion_matrix(y_test, xgb_test_default_predict))
print(classification_report(y_test, xgb_test_default_predict))

[[409   1   1   2   0   0]
 [  2 459   0   0   2   0]
 [  4   0  55   0   0   0]
 [  5   0   0 657   1   0]
 [  1   0   0   0  50   0]
 [  0   0   0   0   0  18]]
              precision    recall  f1-score   support

           0       0.97      0.99      0.98       413
           1       1.00      0.99      0.99       463
           2       0.98      0.93      0.96        59
           3       1.00      0.99      0.99       663
           4       0.94      0.98      0.96        51
           5       1.00      1.00      1.00        18

    accuracy                           0.99      1667
   macro avg       0.98      0.98      0.98      1667
weighted avg       0.99      0.99      0.99      1667



**Hyperparameter Tuning**

In [8]:
learning_rate_list = [0.01, 0.1, 0.5]
max_depth_list = [2, 5, 8]
n_estimators_list = [50, 100, 150]
# reg_lambda = [10, 20, 100]

params_dict = {"learning_rate": learning_rate_list,
               "max_depth": max_depth_list,
               "n_estimators": n_estimators_list}

num_combinations = 1
for v in params_dict.values(): num_combinations *= len(v) 

print(num_combinations)
params_dict

27


{'learning_rate': [0.01, 0.1, 0.5],
 'max_depth': [2, 5, 8],
 'n_estimators': [50, 100, 150]}

In [10]:
# Custom cross-validation method
cv_method = RepeatedStratifiedKFold(n_splits=5, 
                                    n_repeats=3, 
                                    random_state=2022)

model_xgboost_tuning = RandomizedSearchCV(estimator=xgb.XGBClassifier(),
                                param_distributions = params_dict,
                                cv=cv_method,
                                scoring='f1_weighted',
                                return_train_score=True,
                                verbose=2,
                                random_state=2022)

model_xgboost_tuning.fit(X, y)

Fitting 15 folds for each of 10 candidates, totalling 150 fits
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  16.0s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  14.6s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  14.4s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  17.3s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  16.7s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  17.3s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  15.0s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  13.9s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  15.6s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  15.2s
[CV] END ..learning_rate=0.01, max_depth=8, n_estimators=100; total time=  16.0s
[CV] END ..learning_rate=0.01, max_depth=8, n_

RandomizedSearchCV(cv=RepeatedStratifiedKFold(n_repeats=3, n_splits=5, random_state=2022),
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, gamma=None,
                                           gpu_id=None, grow_policy=None,
                                           importance_type=No...
                                           min_child_weight=None, missing=nan,
                                           monotone_constraints=None,
                                           n_estimators=100, n_jobs=None,
                                           num_parallel_tree=None,
                                           predictor=None, random_state=None,
                                           reg_alpha=None, reg_lambda=None, ...),
                   param_distributions={'learning_rate': [0.01, 0.1, 0.5],
                                        'max_depth': [2, 5, 8],
                                        'n_estimators': [50, 100, 150]},
                   random_state=2022, return_train_score=True,
                   scoring='f1_weighted', verbose=2)

In [15]:
model_xgboost_tuning.best_params_

{'n_estimators': 100, 'max_depth': 2, 'learning_rate': 0.5}

In [11]:
df_cv_results = pd.DataFrame(model_xgboost_tuning.cv_results_)
df_cv_results = df_cv_results[['rank_test_score','mean_test_score','mean_train_score',
                               'param_learning_rate', 'param_max_depth', 'param_n_estimators']]
df_cv_results.sort_values(by='rank_test_score', inplace=True)
df_cv_results

,rank_test_score,mean_test_score,mean_train_score,param_learning_rate,param_max_depth,param_n_estimators
4,1,0.991138,1.000000,0.5,2,100
3,2,0.990892,1.000000,0.5,8,150
9,3,0.990834,1.000000,0.5,5,50
7,4,0.988734,0.992200,0.1,2,100
1,5,0.988197,0.999450,0.1,8,50
5,6,0.988196,0.997681,0.1,5,50
0,7,0.984831,0.992646,0.01,8,100
6,8,0.983992,0.990817,0.01,5,150
2,9,0.983095,0.988915,0.01,5,100
8,10,0.974850,0.976702,0.01,2,150


## Final Model
Using best parameters from above step

In [12]:
import time
from sklearn.metrics import f1_score

model_xgboost_fin = xgb.XGBClassifier(**model_xgboost_tuning.best_params_)
start_time = time.time()

model_xgboost_fin.fit(X_train, y_train, verbose =True)
y_pred_test = model_xgboost_fin.predict(X_test)

time_taken = time.time() - start_time
print("Total time taken for the program execution", time_taken) # seconds
print(classification_report(y_test, y_pred_test, digits=5))
# roc_auc_score(y_test, y_pred_test)
f1_score(y_test, y_pred_test, average='weighted')

Total time taken for the program execution 4.327378273010254
              precision    recall  f1-score   support

           0    0.97156   0.99274   0.98204       413
           1    1.00000   0.99568   0.99784       463
           2    0.98214   0.93220   0.95652        59
           3    0.99697   0.99095   0.99395       663
           4    0.98077   1.00000   0.99029        51
           5    1.00000   0.94444   0.97143        18

    accuracy                        0.99040      1667
   macro avg    0.98857   0.97600   0.98201      1667
weighted avg    0.99053   0.99040   0.99040      1667



0.9903970739112765

In [13]:
var_colums = [c for c in X_train.columns if c not in ['state']]

df_var_imp = pd.DataFrame({"Variable": var_colums,
                           "Importance": model_xgboost_fin.feature_importances_})\
                        .sort_values(by='Importance', ascending=False)
df_var_imp[:10]

,Variable,Importance
36,tweets_14,0.339479
41,tweets_19,0.080444
168,description_46,0.065323
4,favourites_count,0.055654
6,default_profile,0.052490
45,tweets_23,0.047831
53,tweets_31,0.044222
16,following_to_followers_ratio,0.041337
9,profile_use_background_image,0.032003
81,tweets_59,0.025980
